## Apply all Cell-Health Models to Training and Testing Sets

**Gregory Way, 2019**

In [1]:
import os
import pandas as pd
from joblib import load

from scripts.ml_utils import load_train_test, load_models

In [2]:
%matplotlib inline

In [3]:
def apply_model(model, feature, train_x, test_x):
    """
    Apply model to training and testing matrix
    """
    pred_train_df = (
        pd.DataFrame(model.predict(train_x), columns=["score"])
        .assign(profiles=train_x.index,
                Metadata_data_type="train",
                model=feature)
    )
    pred_test_df = (
        pd.DataFrame(model.predict(test_x), columns=["score"])
        .assign(profiles=test_x.index,
                Metadata_data_type="test",
                model=feature)
    )

    pred_df = pd.concat([pred_train_df, pred_test_df]).reset_index(drop=True)
    return pred_df

def sample_squared_error(scores, y):
    """
    Calculate the squared error per sample depending on model scores
    """
    metadata_cols = [x for x in scores.columns if x.startswith("Metadata_")]
    scores_values = scores.drop(metadata_cols, axis="columns")
    
    all_squared_error = {}
    for cell_health_feature in scores_values.columns:
        y_subset_df = y.loc[:, cell_health_feature].dropna().T
        scores_subset = scores_values.loc[:, cell_health_feature].reindex(y_subset_df.index).T

        squared_error = (y_subset_df - scores_subset) ** 2
        all_squared_error[cell_health_feature] = squared_error
    
    return pd.DataFrame(all_squared_error).reindex(scores.index)

## 1) Load Models and Model Coefficients

For real data and shuffled model data.

In [4]:
model_dict, model_coef = load_models()
shuffle_model_dict, shuffle_model_coef = load_models(shuffle=True)

In [5]:
# Load Metadata Mapping File
data_dir = os.path.join("..", "1.generate-profiles", "data")
file = os.path.join(data_dir, "profile_id_metadata_mapping.tsv")
metadata_df = pd.read_csv(file, sep='\t')

metadata_df.head()

,Metadata_profile_id,Metadata_cell_line,Metadata_pert_name
0,profile_0,A549,AKT1-1
1,profile_1,A549,AKT1-2
2,profile_2,A549,ARID1B-1
3,profile_3,A549,ARID1B-2
4,profile_4,A549,ATF4-1


## 2) Load Training and Testing Data

In [6]:
x_train_df, x_test_df, y_train_df, y_test_df = load_train_test(drop_metadata=True)

## 3) Output Model Coefficients

In [7]:
# Extract all model coefficients and output to file
coef_df = pd.DataFrame(model_coef)
coef_df.index = x_test_df.columns
coef_df.index.name = "features"

file = os.path.join("results", "all_model_coefficients.tsv")
coef_df.to_csv(file, sep='\t', index=True)

print(coef_df.shape)
coef_df.head(2)

(1278, 70)


,cc_cc_g1_mean,cc_mitosis_ph3_neg_n_spots_mean,cc_edu_pos_high_n_spots_h2ax_mean,cc_polyploid_n_spots_mean,vb_live_cell_roundness,cc_g2_ph3_neg_n_spots_mean,cc_polyploid_high_n_spots_h2ax_mean,vb_ros_mean,cc_cc_n_spots_per_nucleus_area_mean,cc_mitosis_ph3_neg_high_n_spots_h2ax_mean,...,cc_all_nucleus_area_mean,cc_cc_n_spots_mean,vb_ros_back_mean,cc_g1_n_objects,vb_percent_all_apoptosis,cc_all_high_n_spots_h2ax_mean,cc_all_large_round_polyploid_mean,cc_edu_pos_n_objects,cc_all_n_objects,cc_g1_n_spots_per_nucleus_area_mean
features,,,,,,,,,,,,,,,,,,,,,
Cells_AreaShape_Area,-0.0,-0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,-0.0,...,-0.0,0.0,-0.0,0.0,-0.0,-0.0,-0.0,0.000000,0.000000,0.000000
Cells_AreaShape_Center_X,-0.0,0.0,-0.0,0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,...,0.0,-0.0,0.0,0.0,-0.0,-0.0,-0.0,0.006541,0.007159,-0.005849


In [8]:
# Extract all model coefficients and output to file
shuffle_coef_df = pd.DataFrame(shuffle_model_coef)
shuffle_coef_df.index = x_test_df.columns
shuffle_coef_df.index.name = "features"

file = os.path.join("results", "all_model_coefficients_shuffled.tsv")
shuffle_coef_df.to_csv(file, sep='\t', index=True)

print(shuffle_coef_df.shape)
shuffle_coef_df.head(2)

(1278, 70)


,cc_mitosis_ph3_pos_n_spots_mean,vb_percent_caspase_dead_only,vb_infection_percentage,cc_g2_ph3_pos_n_objects,cc_g2_ph3_pos_high_n_spots_h2ax_mean,cc_all_nucleus_roundness_mean,cc_edu_pos_n_spots_mean,vb_percent_all_early_apoptosis,vb_live_cell_width_length,cc_cc_ph3_pos_hoechst_mitosis_mean,...,cc_polynuclear_n_spots_mean,cc_edu_pos_n_objects,cc_g1_n_objects,cc_all_n_objects,cc_cc_g2_ph3_neg_mean,cc_g1_high_n_spots_h2ax_mean,cc_polynuclear_n_spots_per_nucleus_area_mean,cc_polynuclear_n_objects,cc_mitosis_ph3_pos_high_n_spots_h2ax_mean,cc_g2_ph3_neg_n_objects
features,,,,,,,,,,,,,,,,,,,,,
Cells_AreaShape_Area,-0.0,0.0,-0.0,-0.0,-0.0,0.0,-0.0,-0.0,0.0,0.0,...,-0.0,-0.0,-0.0,0.0,0.0,-0.0,-0.0,0.0,-0.0,0.0
Cells_AreaShape_Center_X,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,...,-0.0,0.0,-0.0,0.0,-0.0,0.0,-0.0,-0.0,0.0,-0.0


## 4) Apply all models

For real and shuffled data.

In [9]:
all_scores = []
all_shuffle_scores = []
for cell_health_feature in model_dict.keys():
    # Apply Real Model Classifiers
    model_clf = model_dict[cell_health_feature]
    pred_df = apply_model(model=model_clf,
                          feature=cell_health_feature,
                          train_x=x_train_df,
                          test_x=x_test_df)
    all_scores.append(pred_df)
    
    # Apply Shuffled Model Classifiers
    shuffle_model_clf = shuffle_model_dict[cell_health_feature]
    shuffle_pred_df = apply_model(model=shuffle_model_clf,
                                  feature=cell_health_feature,
                                  train_x=x_train_df,
                                  test_x=x_test_df)
    all_shuffle_scores.append(shuffle_pred_df)

## 5) Concatenate scores with Metadata

In [10]:
# Concatenate real data scores
all_scores = (
    pd.concat(all_scores)
    .reset_index(drop=True)
    .pivot_table(index=["profiles", "Metadata_data_type"],
                 columns="model",
                 values="score")
    .reset_index()
)

all_scores = (
    metadata_df.merge(all_scores,
                      left_on="Metadata_profile_id",
                      right_on="profiles")
    .drop("profiles", axis="columns")
)

all_scores.index = all_scores.Metadata_profile_id
all_scores = all_scores.drop("Metadata_profile_id", axis="columns")

# Output file
file = os.path.join("results", "all_model_predictions.tsv")
all_scores.to_csv(file, sep='\t', index=True)

print(all_scores.shape)
all_scores.head(2)

(357, 73)


,Metadata_cell_line,Metadata_pert_name,Metadata_data_type,cc_all_high_n_spots_h2ax_mean,cc_all_large_notround_polynuclear_mean,cc_all_large_round_polyploid_mean,cc_all_n_objects,cc_all_n_spots_mean,cc_all_n_spots_per_nucleus_area_mean,cc_all_nucleus_area_mean,...,vb_num_live_cells,vb_percent_all_apoptosis,vb_percent_all_early_apoptosis,vb_percent_all_late_apoptosis,vb_percent_caspase_dead_only,vb_percent_dead,vb_percent_dead_only,vb_percent_live,vb_ros_back_mean,vb_ros_mean
Metadata_profile_id,,,,,,,,,,,,,,,,,,,,,
profile_0,A549,AKT1-1,train,0.203774,0.479933,-0.010621,0.204901,0.299410,0.235718,0.023463,...,0.058842,0.174526,0.099569,0.118608,0.257895,0.091457,0.155425,-0.092212,0.212364,0.430335
profile_1,A549,AKT1-2,train,0.044913,0.499595,0.115516,0.320881,0.027806,0.030007,-0.263200,...,0.096353,0.169740,0.073348,0.195297,0.243458,0.279027,0.251313,-0.280351,0.247281,0.533782


In [11]:
# Concatenate shuffled data scores
all_shuffle_scores = (
    pd.concat(all_shuffle_scores)
    .reset_index(drop=True)
    .pivot_table(index=["profiles", "Metadata_data_type"],
                 columns="model",
                 values="score")
    .reset_index()
)

all_shuffle_scores = (
    metadata_df.merge(all_shuffle_scores,
                      left_on="Metadata_profile_id",
                      right_on="profiles")
    .drop("profiles", axis="columns")
)

all_shuffle_scores.index = all_shuffle_scores.Metadata_profile_id
all_shuffle_scores = all_shuffle_scores.drop("Metadata_profile_id", axis="columns")

# Output file
file = os.path.join("results", "all_model_predictions_shuffled.tsv")
all_shuffle_scores.to_csv(file, sep='\t', index=True)

print(all_shuffle_scores.shape)
all_shuffle_scores.head(2)

(357, 73)


,Metadata_cell_line,Metadata_pert_name,Metadata_data_type,cc_all_high_n_spots_h2ax_mean,cc_all_large_notround_polynuclear_mean,cc_all_large_round_polyploid_mean,cc_all_n_objects,cc_all_n_spots_mean,cc_all_n_spots_per_nucleus_area_mean,cc_all_nucleus_area_mean,...,vb_num_live_cells,vb_percent_all_apoptosis,vb_percent_all_early_apoptosis,vb_percent_all_late_apoptosis,vb_percent_caspase_dead_only,vb_percent_dead,vb_percent_dead_only,vb_percent_live,vb_ros_back_mean,vb_ros_mean
Metadata_profile_id,,,,,,,,,,,,,,,,,,,,,
profile_0,A549,AKT1-1,train,0.314977,0.280632,0.261533,-0.160217,0.138126,0.261193,0.032045,...,-0.259548,0.048451,0.169290,0.231602,0.230865,0.174442,0.267044,-0.223782,-0.007581,0.031485
profile_1,A549,AKT1-2,train,0.276184,0.280632,0.261533,-0.160217,0.134277,0.215758,0.074227,...,-0.254897,0.140533,0.167083,0.231602,0.247143,0.105944,0.267044,-0.238984,-0.007581,0.022339


## 6) Calculate the Squared Error of Individual Samples

For real and shuffled data

In [12]:
y_df = pd.concat([y_train_df, y_test_df]).reindex(all_scores.index)

print(y_df.shape)
y_df.head(2)

(357, 70)


,cc_all_high_n_spots_h2ax_mean,cc_all_large_notround_polynuclear_mean,cc_all_large_round_polyploid_mean,cc_all_n_objects,cc_all_n_spots_mean,cc_all_n_spots_per_nucleus_area_mean,cc_all_nucleus_area_mean,cc_all_nucleus_roundness_mean,cc_cc_edu_pos_mean,cc_cc_g1_mean,...,vb_num_live_cells,vb_percent_all_apoptosis,vb_percent_all_early_apoptosis,vb_percent_all_late_apoptosis,vb_percent_caspase_dead_only,vb_percent_dead,vb_percent_dead_only,vb_percent_live,vb_ros_back_mean,vb_ros_mean
Metadata_profile_id,,,,,,,,,,,,,,,,,,,,,
profile_0,-0.005795,0.580351,0.013975,0.381958,0.150696,0.162511,-0.167603,0.040322,0.091643,0.043915,...,0.438339,0.059414,0.082424,0.000000,-0.065050,-0.020236,-0.007970,0.020263,0.408214,0.654575
profile_1,0.050169,1.277730,0.241808,0.577422,0.220829,0.366989,-0.278044,-0.182571,0.893453,-1.023968,...,0.067568,0.256141,0.132834,0.386327,0.575026,0.225091,0.220461,-0.224965,0.284962,0.567898


In [13]:
all_score_error = sample_squared_error(scores=all_scores, y=y_df)

all_score_error = (
    metadata_df.merge(all_score_error,
                      left_on="Metadata_profile_id",
                      right_index=True)
)

# Output file
file = os.path.join("results", "all_model_sample_squared_error.tsv")
all_score_error.to_csv(file, sep='\t', index=False)

print(all_score_error.shape)
all_score_error.head(2)

(357, 73)


,Metadata_profile_id,Metadata_cell_line,Metadata_pert_name,cc_all_high_n_spots_h2ax_mean,cc_all_large_notround_polynuclear_mean,cc_all_large_round_polyploid_mean,cc_all_n_objects,cc_all_n_spots_mean,cc_all_n_spots_per_nucleus_area_mean,cc_all_nucleus_area_mean,...,vb_num_live_cells,vb_percent_all_apoptosis,vb_percent_all_early_apoptosis,vb_percent_all_late_apoptosis,vb_percent_caspase_dead_only,vb_percent_dead,vb_percent_dead_only,vb_percent_live,vb_ros_back_mean,vb_ros_mean
0,profile_0,A549,AKT1-1,0.043919,0.010084,0.000605,0.031349,0.022116,0.005359,0.036506,...,0.144018,0.013251,0.000294,0.014068,0.104293,0.012475,0.026698,0.012651,0.038357,0.050283
1,profile_1,A549,AKT1-2,0.000028,0.605495,0.015950,0.065813,0.037258,0.113557,0.000220,...,0.000829,0.007465,0.003539,0.036493,0.109938,0.002909,0.000952,0.003068,0.001420,0.001164


In [14]:
all_shuffle_score_error = sample_squared_error(scores=all_shuffle_scores, y=y_df)

all_shuffle_score_error = (
    metadata_df.merge(all_shuffle_score_error,
                      left_on="Metadata_profile_id",
                      right_index=True)
)

# Output file
file = os.path.join("results", "all_model_sample_squared_error_shuffled.tsv")
all_shuffle_score_error.to_csv(file, sep='\t', index=False)

print(all_shuffle_score_error.shape)
all_shuffle_score_error.head()

(357, 73)


,Metadata_profile_id,Metadata_cell_line,Metadata_pert_name,cc_all_high_n_spots_h2ax_mean,cc_all_large_notround_polynuclear_mean,cc_all_large_round_polyploid_mean,cc_all_n_objects,cc_all_n_spots_mean,cc_all_n_spots_per_nucleus_area_mean,cc_all_nucleus_area_mean,...,vb_num_live_cells,vb_percent_all_apoptosis,vb_percent_all_early_apoptosis,vb_percent_all_late_apoptosis,vb_percent_caspase_dead_only,vb_percent_dead,vb_percent_dead_only,vb_percent_live,vb_ros_back_mean,vb_ros_mean
0,profile_0,A549,AKT1-1,0.102895,0.089832,0.061285,0.293954,0.000158,0.009738,0.039859,...,0.487047,0.000120,0.007546,0.053640,0.087566,0.037900,0.075633,0.059558,0.172885,0.388241
1,profile_1,A549,AKT1-2,0.051083,0.994205,0.000389,0.544112,0.007491,0.022871,0.124095,...,0.103984,0.013365,0.001173,0.023940,0.107508,0.014196,0.002170,0.000197,0.085581,0.297635
2,profile_2,A549,ARID1B-1,0.030767,0.842821,0.009291,0.240383,0.015782,0.105912,0.167131,...,0.128549,0.020873,0.010510,0.002667,0.084131,0.001593,0.048113,0.007248,0.126868,0.055861
3,profile_3,A549,ARID1B-2,0.097614,0.001328,0.123070,0.099555,0.376113,0.218895,0.073192,...,0.285520,0.016528,0.000270,0.014339,0.088127,0.034908,0.124399,0.021661,0.042381,0.043146
4,profile_4,A549,ATF4-1,14.534840,0.074619,9.137624,4.353669,8.388430,8.726429,4.012598,...,5.061180,0.519710,0.058246,0.001500,0.178908,0.289270,0.026305,0.170984,0.645918,0.030536
